In [1]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [2]:
latent_dim = 32
height = 32
width = 32
channels = 3

In [3]:
generator_input = keras.Input(shape=(latent_dim))

In [4]:
x = layers.Dense(128 * 16 * 16)(generator_input)

In [5]:
x = layers.LeakyReLU()(x)

In [6]:
x = layers.Reshape((16 , 16 , 128))(x)

In [8]:
x = layers.Conv2D(256 , 5 , padding='same')(x)

In [10]:
x = layers.LeakyReLU()(x)

In [11]:
x = layers.Conv2DTranspose(256 , 4 , strides = 2, padding='same')(x)

In [12]:
x = layers.LeakyReLU()(x)

In [13]:
x = layers.Conv2D(256 , 5, padding='same')(x)

In [14]:
x = layers.LeakyReLU()(x)

In [15]:
x = layers.Conv2D(256 , 5 , padding='same')(x)

In [17]:
x = layers.LeakyReLU()(x)

In [18]:
x = layers.Conv2D(channels , 7 , activation='tanh' , padding='same')(x)

In [19]:
generator = keras.models.Model(generator_input , x)

In [20]:
generator.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
dense (Dense)                (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32768)             0         
_________________________________________________________________
reshape (Reshape)            (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 32, 32, 256)       104883

In [21]:
discriminator_input = layers.Input(shape=(height , width , channels))

In [22]:
x = layers.Conv2D(128 , 3)(discriminator_input)

In [23]:
x = layers.LeakyReLU()(x)

In [24]:
x = layers.Conv2D(128 , 4 , strides=2)(x)

In [26]:
x = layers.LeakyReLU()(x)

In [27]:
x = layers.Conv2D(128 , 4 , strides=2)(x)

In [28]:
x = layers.LeakyReLU()(x)

In [29]:
x = layers.Flatten()(x)

In [30]:
x = layers.Dropout(0.4)(x)

In [31]:
x = layers.Dense(1 , activation='sigmoid')(x)

In [33]:
discriminator = keras.models.Model(discriminator_input , x)
discriminator.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 6, 6, 128)         0   

In [35]:
discriminator_optimizer = keras.optimizers.RMSprop(
 learning_rate = 0.0008,
 clipvalue=1.0,
 decay=1e-8
)

In [36]:
discriminator.compile(optimizer=discriminator_optimizer , loss='binary_crossentropy')

In [37]:
discriminator.trainable = False

In [38]:
gan_input = keras.Input(shape=(latent_dim,))

In [40]:
gan_output = discriminator(generator(gan_input))

In [41]:
gan = keras.models.Model(gan_input , gan_output)

In [43]:
gan_optimizer = keras.optimizers.RMSprop(learning_rate=0.0004 , clipvalue=1.0 , decay=1e-8)

In [44]:
gan.compile(optimizer=gan_optimizer , loss='binary_crossentropy')

In [45]:
import os
from tensorflow.keras.preprocessing import image

In [47]:
(train_x , train_y) , (test_x , test_y) = keras.datasets.cifar10.load_data()

170508288/170498071 [==============================] - 207s 1us/step


In [48]:
train_x = train_x[train_y.flatten() == 6]

In [50]:
train_x = train_x.reshape(
(train_x.shape[0],) + (height , width , channels)).astype('float32') / 255

In [51]:
iterations = 10000

In [58]:
import cv2

In [52]:
batch_size = 24

In [53]:
save_dir='saved_image'

In [ ]:
start = 0
for step in range(iterations):
    random_latent_vectors = np.random.normal(size=(batch_size,latent_dim))
    generated_images = generator.predict(random_latent_vectors)
    stop = start + batch_size
    real_images = train_x[start:stop]
    combined_images = np.concatenate([generated_images , real_images])
    
    labels = np.concatenate([np.ones((batch_size , 1)),
                            np.zeros((batch_size , 1))])
    labels += 0.05 * np.random.random(labels.shape)
    
    d_loss = discriminator.train_on_batch(combined_images , labels)
    
    random_latent_vectors = np.random.normal(size=(batch_size,latent_dim))
    
    misleading_targets = np.zeros((batch_size,1))
    
    a_loss = gan.train_on_batch(random_latent_vectors,misleading_targets)
    
    start += batch_size
    if start > len(train_x) - batch_size:
        start = 0
        
    if step % 100 == 0:
        gan.save_weights('gan.h5')
        print('discriminator loss' , d_loss)
        print('adversarial loss' , a_loss)
        
        img = generated_images[0] * 255
        #img.save(os.path.join(save_dir,'generated_frog' + str(step) + '.png'))
        cv2.imwrite(os.path.join(save_dir , 'generated_frog' + str(step) + '.png') , img)
        
        
        
        img = real_images[0] * 255
        cv2.imwrite(os.path.join(save_dir , 'real_frog' + str(step) + '.png') , img)

        
        #img.save(os.path.join(save_dir ,'real_frog' + str(step) + '.png'))

discriminator loss 0.6715965270996094
adversarial loss 0.6390770673751831
discriminator loss 0.6957468390464783
adversarial loss 0.7342211604118347
discriminator loss 0.6951553225517273
adversarial loss 0.765250027179718
discriminator loss 0.6921150088310242
adversarial loss 0.7447593808174133
discriminator loss 0.6863070130348206
adversarial loss 0.7658658027648926
discriminator loss 0.6951813697814941
adversarial loss 0.7464286684989929
discriminator loss 0.6905404925346375
adversarial loss 0.8428745269775391
discriminator loss 0.6919412016868591
adversarial loss 0.7450456023216248
discriminator loss 0.6960576176643372
adversarial loss 0.7019002437591553
discriminator loss 0.6923804879188538
adversarial loss 0.7590726017951965
discriminator loss 0.6958054900169373
adversarial loss 0.7743330597877502
discriminator loss 0.6969180107116699
adversarial loss 1.01005220413208
discriminator loss 0.7406126856803894
adversarial loss 0.7019723057746887
discriminator loss 0.7055750489234924
adv